In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    StackingRegressor
)
from sklearn.metrics import mean_absolute_error, r2_score
import yfinance as yf
from datetime import datetime, timedelta

# Time series models
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# -------------------------------
# 1. Download Stock Data (AAPL)
# -------------------------------
ticker = "IBM"
start = "2022-01-01"
end = datetime.today().strftime('%Y-%m-%d')

df = yf.download(ticker, start=start, end=end)

# Flatten possible MultiIndex columns
df.columns = df.columns.get_level_values(0)

# Keep OHLC
df = df.loc[:, ["Open", "High", "Low", "Close"]].reset_index()

# Create target: Next day's Close
df["Next_Close"] = df["Close"].shift(-1)
df = df.dropna().reset_index(drop=True)

# -------------------------------
# 2. Features & Preprocessor
# -------------------------------
features = ["Open", "High", "Low", "Close"]
X = df[features]
y = df["Next_Close"]

preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), features)],
    remainder="passthrough"
)

# -------------------------------
# 3. Candidate ML Models
# -------------------------------
lr = LinearRegression()
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.001,max_iter=10000)
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
gb = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)

voting_reg = VotingRegressor(estimators=[("lr", lr), ("rf", rf), ("gb", gb)])
weighted_voting_reg = VotingRegressor(
    estimators=[("lr", lr), ("rf", rf), ("gb", gb)],
    weights=[1, 2, 2]
)
stacking_reg = StackingRegressor(
    estimators=[("lr", lr), ("rf", rf), ("gb", gb)],
    final_estimator=Ridge(alpha=1.0)
)

models = {
    "Linear Regression": lr,
    "Ridge Regression": ridge,
    "Lasso Regression": lasso,
    "Random Forest": rf,
    "Gradient Boosting": gb,
    "Voting": voting_reg,
    "Weighted Voting": weighted_voting_reg,
    "Stacking": stacking_reg
}

# -------------------------------
# 4. Evaluate ML Models
# -------------------------------
tscv = TimeSeriesSplit(n_splits=5)
results = {}

for name, model in models.items():
    mae_scores, r2_scores = [], []
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))
    
    results[name] = {"MAE": np.mean(mae_scores), "R²": np.mean(r2_scores)}

# -------------------------------
# 5. Evaluate ARIMA
# -------------------------------
try:
    arima_model = ARIMA(df["Close"], order=(5, 1, 0))
    arima_fit = arima_model.fit()
    arima_forecast = arima_fit.forecast(steps=1)[0]
    results["ARIMA"] = {"MAE": np.nan, "R²": np.nan}
except Exception as e:
    print("ARIMA failed:", e)
    arima_forecast = None

# -------------------------------
# 6. Evaluate Prophet
# -------------------------------
try:
    prophet_df = df[["Date", "Close"]].rename(columns={"Date": "ds", "Close": "y"})
    prophet = Prophet(daily_seasonality=True)
    prophet.fit(prophet_df)
    future = prophet.make_future_dataframe(periods=1)
    forecast = prophet.predict(future)
    prophet_forecast = forecast.iloc[-1]["yhat"]
    results["Prophet"] = {"MAE": np.nan, "R²": np.nan}
except Exception as e:
    print("Prophet failed:", e)
    prophet_forecast = None

# -------------------------------
# 7. Results
# -------------------------------
results_df = pd.DataFrame(results).T.sort_values(by="MAE", na_position="last")
print("\nModel Performance (Cross-Validation):")
print(results_df)

# -------------------------------
# 8. AutoML: Pick Best ML Model
# -------------------------------
best_model_name = results_df.dropna().index[0]
print(f"\n✅ Best ML Model Selected: {best_model_name}")

best_model = models[best_model_name]
final_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", best_model)])
final_pipeline.fit(X, y)

# -------------------------------
# 9. Predict Tomorrow’s Price
# -------------------------------
latest_features = df[features].iloc[-1:]
next_day_prediction = final_pipeline.predict(latest_features)[0]

print(f"\n📈 Predicted next close (ML AutoML) for {ticker} (tomorrow): {next_day_prediction:.2f}")

if arima_forecast is not None:
    print(f"📉 ARIMA Forecast: {arima_forecast:.2f}")

if prophet_forecast is not None:
    print(f"🔮 Prophet Forecast: {prophet_forecast:.2f}")


d:\new_stock_prediction\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\INDIA\AppData\Local\Temp\ipykernel_120232\1895685840.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed


ARIMA failed: 0


20:09:31 - cmdstanpy - INFO - Chain [1] start processing
20:09:32 - cmdstanpy - INFO - Chain [1] done processing



Model Performance (Cross-Validation):
                         MAE        R²
Stacking            1.849238  0.964358
Linear Regression   1.883032  0.963754
Lasso Regression    1.890782  0.963507
Ridge Regression    1.946420  0.962279
Voting             10.397996  0.237130
Weighted Voting    12.299397 -0.076268
Random Forest      14.971265 -0.620751
Gradient Boosting  15.358269 -0.684726
Prophet                  NaN       NaN

✅ Best ML Model Selected: Stacking

📈 Predicted next close (ML AutoML) for IBM (tomorrow): 237.95
🔮 Prophet Forecast: 271.79


In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    StackingRegressor
)
from sklearn.metrics import mean_absolute_error, r2_score
import yfinance as yf
from datetime import datetime

# -------------------------------
# 1. Download Stock Data (e.g., GOOG)
# -------------------------------
ticker = "IBM"
start = "2020-01-01"
end = datetime.today().strftime('%Y-%m-%d')

df = yf.download(ticker, start=start, end=end)

# Flatten possible MultiIndex columns
df.columns = df.columns.get_level_values(0)

# Keep OHLC
df = df.loc[:, ["Open", "High", "Low", "Close"]].reset_index()

# Create target: Next day's Close
df["Next_Close"] = df["Close"].shift(-1)
df = df.dropna().reset_index(drop=True)

# -------------------------------
# 2. Features & Preprocessor
# -------------------------------
features = ["Open", "High", "Low", "Close"]
X = df[features]
y = df["Next_Close"]

preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), features)],
    remainder="drop"
)

# -------------------------------
# 3. Candidate ML Models
# -------------------------------
lr = LinearRegression()
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.001, max_iter=30000)
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
gb = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)

voting_reg = VotingRegressor(estimators=[("lr", lr), ("rf", rf), ("gb", gb)])
weighted_voting_reg = VotingRegressor(
    estimators=[("lr", lr), ("rf", rf), ("gb", gb)],
    weights=[1, 2, 2]
)
stacking_reg = StackingRegressor(
    estimators=[("lr", lr), ("rf", rf), ("gb", gb)],
    final_estimator=Ridge(alpha=1.0)
)

models = {
    "Linear Regression": lr,
    "Ridge Regression": ridge,
    "Lasso Regression": lasso,
    "Random Forest": rf,
    "Gradient Boosting": gb,
    "Voting": voting_reg,
    "Weighted Voting": weighted_voting_reg,
    "Stacking": stacking_reg
}

# -------------------------------
# 4. Evaluate ML Models
# -------------------------------
tscv = TimeSeriesSplit(n_splits=5)
results = {}

for name, model in models.items():
    mae_scores, r2_scores = [], []
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))
    
    results[name] = {"MAE": np.mean(mae_scores), "R²": np.mean(r2_scores)}

# -------------------------------
# 5. Results
# -------------------------------
results_df = pd.DataFrame(results).T.sort_values(by="MAE", na_position="last")
print("\nModel Performance (Cross-Validation):")
print(results_df)

# -------------------------------
# 6. AutoML: Pick Best ML Model
# -------------------------------
best_model_name = results_df.dropna().index[0]
print(f"\n✅ Best ML Model Selected: {best_model_name}")

best_model = models[best_model_name]
final_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", best_model)])
final_pipeline.fit(X, y)

# -------------------------------
# 7. Save Model as {ticker}_{today}.joblib
# -------------------------------
today_str = datetime.today().strftime("%Y%m%d")
model_filename = f"{ticker}_{today_str}.joblib"
joblib.dump(final_pipeline, model_filename)
print(f"💾 Model saved as {model_filename}")

# -------------------------------
# 8. Predict Tomorrow’s Price
# -------------------------------
latest_features = df[features].iloc[-1:]
next_day_prediction = final_pipeline.predict(latest_features)[0]

print(f"\n📈 Predicted next close (ML AutoML) for {ticker} (tomorrow): {next_day_prediction:.2f}")


C:\Users\INDIA\AppData\Local\Temp\ipykernel_120232\3720922474.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed



Model Performance (Cross-Validation):
                         MAE        R²
Linear Regression   1.637006  0.955742
Lasso Regression    1.646102  0.955605
Ridge Regression    1.660134  0.955096
Stacking            2.222128  0.940552
Voting             10.784782  0.226833
Weighted Voting    12.781682 -0.090284
Random Forest      15.678795 -0.659543
Gradient Boosting  15.903841 -0.688342

✅ Best ML Model Selected: Linear Regression
💾 Model saved as IBM_20250817.joblib

📈 Predicted next close (ML AutoML) for IBM (tomorrow): 237.39
